In [180]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import KFold, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PowerTransformer
import pickle



In [181]:
df=pd.read_csv(r"E:/data science/NBFI project/Test_Dataset.csv")
df.head()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,...,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau
0,12202227,11250,0.0,1.0,1.0,1.0,0.0,112500,4474.8,Relative,...,16.0,Yes,Yes,Self-employed,NaN,0.757151,0.629674,0.0515,716.0,2.0
1,12279381,13500,1.0,1.0,1.0,1.0,3.0,49752,3252.15,Alone,...,18.0,No,Yes,Self-employed,NaN,0.342269,NaN,NaN,181.0,NaN
2,12222714,38250,0.0,1.0,0.0,1.0,0.0,67500,3375,Alone,...,19.0,Yes,Yes,Business Entity Type 3,NaN,0.683665,NaN,0.1503,281.0,NaN
3,12265215,20250,0.0,0.0,1.0,0.0,0.0,50849.55,3814.65,Alone,...,20.0,No,No,Medicine,0.351581,0.22995,0.672243,NaN,1442.0,4.0
4,12203970,13500,1.0,1.0,0.0,1.0,0.0,43603.2,2515.95,Alone,...,13.0,Yes,Yes,Self-employed,0.731110,0.566971,0.730987,NaN,927.0,1.0


In [182]:
df.describe()

,ID,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Family_Members,Cleint_City_Rating,Application_Process_Day,Application_Process_Hour,Score_Source_1,Social_Circle_Default,Phone_Change,Credit_Bureau
count,8.090000e+04,78495.000000,78414.000000,78446.000000,78553.000000,78572.000000,27319.000000,80900.0,80900.000000,80900.000000,79256.000000,79269.000000,79361.000000,78494.000000,35407.000000,39810.000000,78477.000000,68618.000000
mean,1.224045e+07,0.337881,0.332262,0.497833,0.693252,0.418393,12.013727,1.0,0.199802,0.279419,2.153288,2.029709,3.157722,12.070566,0.501752,0.117942,961.007263,1.897432
std,2.335396e+04,0.472991,0.471027,0.499998,0.461147,0.726007,11.937654,0.0,0.399854,0.448717,0.915802,0.501864,1.754763,3.257224,0.211653,0.108621,827.880031,1.867447
min,1.220000e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.015600,0.000000,0.000000,0.000000
25%,1.222023e+07,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,1.0,0.000000,0.000000,2.000000,2.000000,2.000000,10.000000,0.332620,0.057700,271.000000,0.000000
50%,1.224045e+07,0.000000,0.000000,0.000000,1.000000,0.000000,9.000000,1.0,0.000000,0.000000,2.000000,2.000000,3.000000,12.000000,0.506789,0.087100,755.000000,1.000000
75%,1.226068e+07,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000,1.0,0.000000,1.000000,3.000000,2.000000,5.000000,14.000000,0.675585,0.148500,1568.000000,3.000000
max,1.228090e+07,1.000000,1.000000,1.000000,1.000000,19.000000,65.000000,1.0,1.000000,1.000000,20.000000,3.000000,6.000000,23.000000,0.946098,1.000000,4292.000000,25.000000


In [183]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80900 entries, 0 to 80899
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          80900 non-null  int64  
 1   Client_Income               78466 non-null  object 
 2   Car_Owned                   78495 non-null  float64
 3   Bike_Owned                  78414 non-null  float64
 4   Active_Loan                 78446 non-null  float64
 5   House_Own                   78553 non-null  float64
 6   Child_Count                 78572 non-null  float64
 7   Credit_Amount               78613 non-null  object 
 8   Loan_Annuity                77697 non-null  object 
 9   Accompany_Client            79692 non-null  object 
 10  Client_Income_Type          78399 non-null  object 
 11  Client_Education            78550 non-null  object 
 12  Client_Marital_Status       78457 non-null  object 
 13  Client_Gender               792

In [184]:
df.shape

(80900, 39)

In [185]:
df.isnull().sum()

ID                                0
Client_Income                  2434
Car_Owned                      2405
Bike_Owned                     2486
Active_Loan                    2454
House_Own                      2347
Child_Count                    2328
Credit_Amount                  2287
Loan_Annuity                   3203
Accompany_Client               1208
Client_Income_Type             2501
Client_Education               2350
Client_Marital_Status          2443
Client_Gender                  1627
Loan_Contract_Type             2321
Client_Housing_Type            2408
Population_Region_Relative     3270
Age_Days                       2479
Employed_Days                  2421
Registration_Days              2398
ID_Days                        3881
Own_House_Age                 53581
Mobile_Tag                        0
Homephone_Tag                     0
Workphone_Working                 0
Client_Occupation             27845
Client_Family_Members          1644
Cleint_City_Rating          

In [186]:
df = df.drop(["Own_House_Age","Score_Source_1","Social_Circle_Default","Client_Occupation","Score_Source_3"],axis=1)
df

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,...,Client_Family_Members,Cleint_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_2,Phone_Change,Credit_Bureau
0,12202227,11250,0.0,1.0,1.0,1.0,0.0,112500,4474.8,Relative,...,NaN,2.0,4.0,16.0,Yes,Yes,Self-employed,0.757151,716.0,2.0
1,12279381,13500,1.0,1.0,1.0,1.0,3.0,49752,3252.15,Alone,...,5.0,3.0,3.0,18.0,No,Yes,Self-employed,0.342269,181.0,NaN
2,12222714,38250,0.0,1.0,0.0,1.0,0.0,67500,3375,Alone,...,2.0,1.0,2.0,19.0,Yes,Yes,Business Entity Type 3,0.683665,281.0,NaN
3,12265215,20250,0.0,0.0,1.0,0.0,0.0,50849.55,3814.65,Alone,...,1.0,2.0,6.0,20.0,No,No,Medicine,0.22995,1442.0,4.0
4,12203970,13500,1.0,1.0,0.0,1.0,0.0,43603.2,2515.95,Alone,...,2.0,2.0,1.0,13.0,Yes,Yes,Self-employed,0.566971,927.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80895,12210181,11700,0.0,0.0,0.0,1.0,2.0,107820,3152.25,Alone,...,4.0,2.0,2.0,9.0,Yes,No,Self-employed,0.599436,537.0,1.0
80896,12206257,29250,0.0,1.0,1.0,1.0,2.0,116430.75,4630.5,Alone,...,4.0,1.0,6.0,15.0,Yes,Yes,Kindergarten,NaN,316.0,0.0
80897,12200446,54000,1.0,0.0,1.0,1.0,0.0,52128,2742.3,Alone,...,2.0,2.0,3.0,14.0,Yes,No,Self-employed,0.403059,426.0,0.0
80898,12244943,13500,0.0,0.0,1.0,0.0,0.0,25470,1435.05,Alone,...,2.0,2.0,6.0,5.0,Yes,Yes,Industry: type 9,0.236402,1186.0,2.0


In [187]:

print(df["Client_Gender"].value_counts())

Client_Gender
Male      52146
Female    27126
XNA           1
Name: count, dtype: int64


In [188]:
num_columns = df.select_dtypes(include=['int64','float64']).columns

print(num_columns)

Index(['ID', 'Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own',
       'Child_Count', 'Mobile_Tag', 'Homephone_Tag', 'Workphone_Working',
       'Client_Family_Members', 'Cleint_City_Rating',
       'Application_Process_Day', 'Application_Process_Hour', 'Phone_Change',
       'Credit_Bureau'],
      dtype='object')


In [189]:
cat_columns = df.select_dtypes(include='object').columns

print(cat_columns)

Index(['Client_Income', 'Credit_Amount', 'Loan_Annuity', 'Accompany_Client',
       'Client_Income_Type', 'Client_Education', 'Client_Marital_Status',
       'Client_Gender', 'Loan_Contract_Type', 'Client_Housing_Type',
       'Population_Region_Relative', 'Age_Days', 'Employed_Days',
       'Registration_Days', 'ID_Days', 'Client_Permanent_Match_Tag',
       'Client_Contact_Work_Tag', 'Type_Organization', 'Score_Source_2'],
      dtype='object')


In [190]:
df.value_counts()

ID        Client_Income  Car_Owned  Bike_Owned  Active_Loan  House_Own  Child_Count  Credit_Amount  Loan_Annuity  Accompany_Client  Client_Income_Type  Client_Education  Client_Marital_Status  Client_Gender  Loan_Contract_Type  Client_Housing_Type  Population_Region_Relative  Age_Days  Employed_Days  Registration_Days  ID_Days  Mobile_Tag  Homephone_Tag  Workphone_Working  Client_Family_Members  Cleint_City_Rating  Application_Process_Day  Application_Process_Hour  Client_Permanent_Match_Tag  Client_Contact_Work_Tag  Type_Organization       Score_Source_2  Phone_Change  Credit_Bureau
12200004  16200          0.0        0.0         0.0          1.0        0.0          53691.75       2873.7        Alone             Commercial          Secondary         M                      Male           CL                  Home                 0.016612                    23470.0   4737.0         8336.0             4224.0   1           0              1                  2.0                    2.0       

In [191]:
df['Client_Income'] = df['Client_Income'].str.strip('$')
df['Client_Income'].replace(to_replace='', value= None, regex=True, inplace=True)
df['Client_Income']=df['Client_Income'].fillna(df['Client_Income'].median())

In [192]:
df['Loan_Annuity'] = df['Loan_Annuity'].str.strip('$')
df['Loan_Annuity'] = df['Loan_Annuity'].str.strip('#VALUE!')
df['Loan_Annuity'].replace(to_replace='', value= None, regex=True, inplace=True)
df['Loan_Annuity']=df['Loan_Annuity'].fillna(df['Loan_Annuity'].median())


In [193]:
df['Credit_Amount'] = df['Credit_Amount'].str.strip('$')
df['Credit_Amount'].replace(to_replace='', value= None, regex=True, inplace=True)
df['Credit_Amount']=df['Credit_Amount'].fillna(df['Credit_Amount'].median())

In [194]:
df['Population_Region_Relative'] = df['Population_Region_Relative'].str.strip('#')
df['Population_Region_Relative'] = df['Population_Region_Relative'].str.strip('@')
df['Population_Region_Relative'] = df['Population_Region_Relative'].str.strip('x')
df['Population_Region_Relative'].replace(to_replace='', value= None, regex=True, inplace=True)
df['Population_Region_Relative']=df['Population_Region_Relative'].fillna(df['Population_Region_Relative'].median())

In [195]:
df['ID_Days'] = df['ID_Days'].str.strip('x')
df['ID_Days'].replace(to_replace='', value= None, regex=True, inplace=True)
df['ID_Days']=df['ID_Days'].fillna(df['ID_Days'].median())

In [196]:
df['Age_Days'] = df['Age_Days'].str.strip('x')
df['Age_Days'].replace(to_replace='', value= None, regex=True, inplace=True)
df['Age_Days']=df['Age_Days'].fillna(df['Age_Days'].median())

In [197]:
df['Employed_Days'] = df['Employed_Days'].str.strip('x')
df['Employed_Days'].replace(to_replace='', value= None, regex=True, inplace=True)
df['Employed_Days']=df['Employed_Days'].fillna(df['Employed_Days'].median())

In [198]:
df['Registration_Days'] = df['Registration_Days'].str.strip('x')
df['Registration_Days'].replace(to_replace='', value= None, regex=True, inplace=True)
df['Registration_Days']=df['Registration_Days'].fillna(df['Registration_Days'].median())

In [199]:
df['Accompany_Client'] = df['Accompany_Client'].str.strip('##')
df['Accompany_Client'].replace(to_replace='', value= None, regex=True, inplace=True)
df['Accompany_Client']=df['Accompany_Client'].fillna(df['Accompany_Client'].mode()[0])

In [200]:
df['Score_Source_2'].replace(to_replace='#', value= None, regex=True, inplace=True)
df['Score_Source_2']=df['Score_Source_2'].fillna(df['Score_Source_2'].median())

In [201]:
df.isnull().sum()

ID                                0
Client_Income                     0
Car_Owned                      2405
Bike_Owned                     2486
Active_Loan                    2454
House_Own                      2347
Child_Count                    2328
Credit_Amount                     0
Loan_Annuity                      0
Accompany_Client                  0
Client_Income_Type             2501
Client_Education               2350
Client_Marital_Status          2443
Client_Gender                  1627
Loan_Contract_Type             2321
Client_Housing_Type            2408
Population_Region_Relative        0
Age_Days                          0
Employed_Days                     0
Registration_Days                 0
ID_Days                           0
Mobile_Tag                        0
Homephone_Tag                     0
Workphone_Working                 0
Client_Family_Members          1644
Cleint_City_Rating             1631
Application_Process_Day        1539
Application_Process_Hour    

In [202]:
df['Credit_Bureau']=df['Credit_Bureau'].fillna(df['Credit_Bureau'].mode()[0])
df['Client_Income_Type']=df['Client_Income_Type'].fillna(df['Client_Income_Type'].mode()[0])
df['Bike_Owned']=df['Bike_Owned'].fillna(df['Bike_Owned'].mode()[0])
df['Type_Organization']=df['Type_Organization'].fillna(df['Type_Organization'].mode()[0])
df['Active_Loan']=df['Active_Loan'].fillna(df['Active_Loan'].mode()[0])
df['Client_Marital_Status']=df['Client_Marital_Status'].fillna(df['Client_Marital_Status'].mode()[0])
df['Client_Housing_Type']=df['Client_Housing_Type'].fillna(df['Client_Housing_Type'].mode()[0])
df['Application_Process_Hour']=df['Application_Process_Hour'].fillna(df['Application_Process_Hour'].mode()[0])
df['Car_Owned']=df['Car_Owned'].fillna(df['Car_Owned'].mode()[0])
df['Client_Education']=df['Client_Education'].fillna(df['Client_Education'].mode()[0])
df['House_Own']=df['House_Own'].fillna(df['House_Own'].mode()[0])
df['Loan_Contract_Type']=df['Loan_Contract_Type'].fillna(df['Loan_Contract_Type'].mode()[0])
df['Cleint_City_Rating']=df['Cleint_City_Rating'].fillna(df['Cleint_City_Rating'].mode()[0])
df['Client_Gender']=df['Client_Gender'].fillna(df['Client_Gender'].mode()[0])
df['Application_Process_Day']=df['Application_Process_Day'].fillna(df['Application_Process_Day'].mode()[0])
df['Application_Process_Day']=df['Accompany_Client'].fillna(df['Accompany_Client'].mode()[0])
df['Phone_Change']=df['Phone_Change'].fillna(df['Phone_Change'].median())
df['Child_Count']=df['Child_Count'].fillna(df['Child_Count'].median())
df['Client_Family_Members']=df['Client_Family_Members'].fillna(df['Client_Family_Members'].median())

In [203]:
df.isnull().sum()

ID                            0
Client_Income                 0
Car_Owned                     0
Bike_Owned                    0
Active_Loan                   0
House_Own                     0
Child_Count                   0
Credit_Amount                 0
Loan_Annuity                  0
Accompany_Client              0
Client_Income_Type            0
Client_Education              0
Client_Marital_Status         0
Client_Gender                 0
Loan_Contract_Type            0
Client_Housing_Type           0
Population_Region_Relative    0
Age_Days                      0
Employed_Days                 0
Registration_Days             0
ID_Days                       0
Mobile_Tag                    0
Homephone_Tag                 0
Workphone_Working             0
Client_Family_Members         0
Cleint_City_Rating            0
Application_Process_Day       0
Application_Process_Hour      0
Client_Permanent_Match_Tag    0
Client_Contact_Work_Tag       0
Type_Organization             0
Score_So

In [204]:
df.head()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,...,Client_Family_Members,Cleint_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_2,Phone_Change,Credit_Bureau
0,12202227,11250,0.0,1.0,1.0,1.0,0.0,112500,4474.8,Relative,...,2.0,2.0,Relative,16.0,Yes,Yes,Self-employed,0.757151,716.0,2.0
1,12279381,13500,1.0,1.0,1.0,1.0,3.0,49752,3252.15,Alone,...,5.0,3.0,Alone,18.0,No,Yes,Self-employed,0.342269,181.0,0.0
2,12222714,38250,0.0,1.0,0.0,1.0,0.0,67500,3375,Alone,...,2.0,1.0,Alone,19.0,Yes,Yes,Business Entity Type 3,0.683665,281.0,0.0
3,12265215,20250,0.0,0.0,1.0,0.0,0.0,50849.55,3814.65,Alone,...,1.0,2.0,Alone,20.0,No,No,Medicine,0.22995,1442.0,4.0
4,12203970,13500,1.0,1.0,0.0,1.0,0.0,43603.2,2515.95,Alone,...,2.0,2.0,Alone,13.0,Yes,Yes,Self-employed,0.566971,927.0,1.0


In [205]:
(df.isnull().sum()/len(df)).sort_values(ascending = False)

ID                            0.0
Cleint_City_Rating            0.0
Registration_Days             0.0
ID_Days                       0.0
Mobile_Tag                    0.0
Homephone_Tag                 0.0
Workphone_Working             0.0
Client_Family_Members         0.0
Application_Process_Day       0.0
Client_Income                 0.0
Application_Process_Hour      0.0
Client_Permanent_Match_Tag    0.0
Client_Contact_Work_Tag       0.0
Type_Organization             0.0
Score_Source_2                0.0
Phone_Change                  0.0
Employed_Days                 0.0
Age_Days                      0.0
Population_Region_Relative    0.0
Client_Housing_Type           0.0
Loan_Contract_Type            0.0
Client_Gender                 0.0
Client_Marital_Status         0.0
Client_Education              0.0
Client_Income_Type            0.0
Accompany_Client              0.0
Loan_Annuity                  0.0
Credit_Amount                 0.0
Child_Count                   0.0
House_Own     

In [206]:
df['Client_Income'] = df['Client_Income'].astype('float')
df['Credit_Amount'] = df['Credit_Amount'].astype('float')
df['Loan_Annuity'] = df['Loan_Annuity'].astype('float')
df['Population_Region_Relative'] = df['Population_Region_Relative'].astype('float')
df['Age_Days'] = df['Age_Days'].astype('float')
df['Employed_Days'] = df['Employed_Days'].astype('float')
df['Registration_Days'] = df['Registration_Days'].astype('float')
df['ID_Days'] = df['ID_Days'].astype('float')
df['Score_Source_2'] = df['Score_Source_2'].astype('float')
df['Phone_Change'] = df['Phone_Change'].astype('float')

In [207]:
def group(value):
    if value >= 0.8:
        return 0
    elif value >= 0.7  and value < 0.8:
        return 1
    elif value >= 0.6  and value < 0.7:
        return 2
    elif value >= 0.5  and value < 0.6:
        return 3
    elif value >= 0.4  and value < 0.5:
        return 4
    elif value >= 0.3  and value < 0.4:
        return 5
    elif value >= 0.2  and value < 0.3:
        return 6
    elif value >= 0.1  and value < 0.2:
        return 7
    else:
        return 8

In [208]:
df['score_group'] = df.apply(lambda x: group(x['Score_Source_2']),axis=1)

In [209]:
df = df.drop('Score_Source_2', axis = 1)

In [210]:
df = df[df['Client_Gender'] != 'XNA']

In [211]:
categorical_columns=['Accompany_Client', 'Client_Income_Type', 'Client_Education',
       'Client_Marital_Status', 'Client_Gender', 'Loan_Contract_Type',
       'Client_Housing_Type', 'Application_Process_Day',
       'Client_Permanent_Match_Tag', 'Client_Contact_Work_Tag',
       'Type_Organization']

In [212]:
for cat_col in categorical_columns:
    
    columns = {} 
    i   = 0
    for  temp_col in df[cat_col].unique():
        columns[str(temp_col)] = i
        i+=1
    print(cat_col, ":" , columns)
    df["new_{}".format(cat_col)] = df[cat_col].apply(lambda x: columns[str(x)])

Accompany_Client : {'Relative': 0, 'Alone': 1, 'Kids': 2, 'Partner': 3, 'Others': 4, 'Group': 5}
Client_Income_Type : {'Service': 0, 'Commercial': 1, 'Govt Job': 2, 'Retired': 3, 'Student': 4, 'Unemployed': 5, 'Businessman': 6}
Client_Education : {'Secondary': 0, 'Graduation': 1, 'Graduation dropout': 2, 'junior secondary': 3, 'Post Grad': 4}
Client_Marital_Status : {'M': 0, 'S': 1, 'W': 2, 'D': 3, 'Unknown': 4}
Client_Gender : {'Male': 0, 'Female': 1}
Loan_Contract_Type : {'CL': 0, 'RL': 1}
Client_Housing_Type : {'Home': 0, 'Municipal': 1, 'Family': 2, 'Office': 3, 'Rental': 4, 'Shared': 5}
Application_Process_Day : {'Relative': 0, 'Alone': 1, 'Kids': 2, 'Partner': 3, 'Others': 4, 'Group': 5}
Client_Permanent_Match_Tag : {'Yes': 0, 'No': 1}
Client_Contact_Work_Tag : {'Yes': 0, 'No': 1}
Type_Organization : {'Self-employed': 0, 'Business Entity Type 3': 1, 'Medicine': 2, 'XNA': 3, 'Security Ministries': 4, 'Industry: type 11': 5, 'Other': 6, 'Trade: type 3': 7, 'Industry: type 9': 8, 'T

In [213]:
Accompany_Client ={'Alone': 0, 'Relative': 1, 'Others': 2, 'Kids': 3, 'Partner': 4, 'Group': 5}
Client_Income_Type ={'Commercial': 0, 'Service': 1, 'Govt Job': 2, 'Retired': 3, 'Student': 4, 'Maternity leave': 5, 'Unemployed': 6, 'Businessman': 7}
Client_Education={'Secondary': 0, 'Graduation': 1, 'Graduation dropout': 2, 'Junior secondary': 3, 'Post Grad': 4}
Client_Marital_Status ={'M': 0, 'W': 1, 'S': 2, 'D': 3}
Client_Gender = {'Male': 0, 'Female': 1}
Loan_Contract_Type = {'CL': 0, 'RL': 1}
Client_Housing_Type = {'Home': 0, 'Family': 1, 'Municipal': 2, 'Rental': 3, 'Office': 4, 'Shared': 5}
Application_Process_Day ={'Alone': 0, 'Relative': 1, 'Others': 2, 'Kids': 3, 'Partner': 4, 'Group': 5}
Client_Permanent_Match_Tag = {'Yes': 0, 'No': 1}
Client_Contact_Work_Tag = {'Yes': 0, 'No': 1}
Type_Organization = {'Self-employed': 0, 'Government': 1, 'Business Entity Type 3': 2, 'Other': 3, 'Industry: type 3': 4, 'Business Entity Type 2': 5, 'Business Entity Type 1': 6, 'Transport: type 4': 7, 'Construction': 8, 'Kindergarten': 9, 'Trade: type 3': 10, 'Industry: type 2': 11, 'Trade: type 7': 12, 'Trade: type 2': 13, 'Agriculture': 14, 'Military': 15, 'Medicine': 16, 'XNA': 17, 'Housing': 18, 'Industry: type 1': 19, 'Industry: type 11': 20, 'Bank': 21, 'School': 22, 'Industry: type 9': 23, 'Postal': 24, 'University': 25, 'Transport: type 2': 26, 'Restaurant': 27, 'Electricity': 28, 'Police': 29, 'Industry: type 4': 30, 'Security Ministries': 31, 'Services': 32, 'Transport: type 3': 33, 'Mobile': 34, 'Hotel': 35, 'Security': 36, 'Industry: type 7': 37, 'Advertising': 38, 'Cleaning': 39, 'Realtor': 40, 'Trade: type 6': 41, 'Culture': 42, 'Industry: type 5': 43, 'Telecom': 44, 'Trade: type 1': 45, 'Industry: type 12': 46, 'Industry: type 8': 47, 'Insurance': 48, 'Emergency': 49, 'Legal Services': 50, 'Industry: type 10': 51, 'Trade: type 4': 52, 'Industry: type 6': 53, 'Transport: type 1': 54, 'Industry: type 13': 55, 'Religion': 56, 'Trade: type 5': 57}

In [214]:
df['Accompany_Client'] =df['Accompany_Client'].map(Accompany_Client)
df['Client_Income_Type'] =df['Client_Income_Type'].map(Client_Income_Type)
df['Client_Education'] =df['Client_Education'].map(Client_Education)
df['Client_Marital_Status'] =df['Client_Marital_Status'].map(Client_Marital_Status)
df['Client_Gender'] =df['Client_Gender'].map(Client_Gender)
df['Loan_Contract_Type'] =df['Loan_Contract_Type'].map(Loan_Contract_Type)
df['Client_Housing_Type'] =df['Client_Housing_Type'].map(Client_Housing_Type)
df['Application_Process_Day'] =df['Application_Process_Day'].map(Application_Process_Day)
df['Client_Permanent_Match_Tag'] =df['Client_Permanent_Match_Tag'].map(Client_Permanent_Match_Tag)
df['Client_Contact_Work_Tag'] =df['Client_Contact_Work_Tag'].map(Client_Contact_Work_Tag)
df['Type_Organization'] =df['Type_Organization'].map(Type_Organization)

In [215]:
df.isnull().sum()

ID                                  0
Client_Income                       0
Car_Owned                           0
Bike_Owned                          0
Active_Loan                         0
House_Own                           0
Child_Count                         0
Credit_Amount                       0
Loan_Annuity                        0
Accompany_Client                    0
Client_Income_Type                  0
Client_Education                  966
Client_Marital_Status               1
Client_Gender                       0
Loan_Contract_Type                  0
Client_Housing_Type                 0
Population_Region_Relative          0
Age_Days                            0
Employed_Days                       0
Registration_Days                   0
ID_Days                             0
Mobile_Tag                          0
Homephone_Tag                       0
Workphone_Working                   0
Client_Family_Members               0
Cleint_City_Rating                  0
Application_

In [216]:
df['Client_Education']=df['Client_Education'].fillna(df['Client_Education'].mode()[0])
df['Client_Marital_Status']=df['Client_Marital_Status'].fillna(df['Client_Marital_Status'].mode()[0])


In [217]:
df.isnull().sum()

ID                                0
Client_Income                     0
Car_Owned                         0
Bike_Owned                        0
Active_Loan                       0
House_Own                         0
Child_Count                       0
Credit_Amount                     0
Loan_Annuity                      0
Accompany_Client                  0
Client_Income_Type                0
Client_Education                  0
Client_Marital_Status             0
Client_Gender                     0
Loan_Contract_Type                0
Client_Housing_Type               0
Population_Region_Relative        0
Age_Days                          0
Employed_Days                     0
Registration_Days                 0
ID_Days                           0
Mobile_Tag                        0
Homephone_Tag                     0
Workphone_Working                 0
Client_Family_Members             0
Cleint_City_Rating                0
Application_Process_Day           0
Application_Process_Hour    

In [218]:
data=df

In [219]:
df=df.drop('ID',axis=1)
df

,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,...,new_Client_Income_Type,new_Client_Education,new_Client_Marital_Status,new_Client_Gender,new_Loan_Contract_Type,new_Client_Housing_Type,new_Application_Process_Day,new_Client_Permanent_Match_Tag,new_Client_Contact_Work_Tag,new_Type_Organization
0,11250.0,0.0,1.0,1.0,1.0,0.0,112500.00,4474.80,1,1,...,0,0,0,0,0,0,0,0,0,0
1,13500.0,1.0,1.0,1.0,1.0,3.0,49752.00,3252.15,0,1,...,0,0,0,1,0,0,1,1,0,0
2,38250.0,0.0,1.0,0.0,1.0,0.0,67500.00,3375.00,0,0,...,1,1,0,1,1,0,1,0,0,1
3,20250.0,0.0,0.0,1.0,0.0,0.0,50849.55,3814.65,0,2,...,2,0,1,0,0,0,1,1,1,2
4,13500.0,1.0,1.0,0.0,1.0,0.0,43603.20,2515.95,0,1,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80895,11700.0,0.0,0.0,0.0,1.0,2.0,107820.00,3152.25,0,1,...,0,0,0,0,0,0,1,0,1,0
80896,29250.0,0.0,1.0,1.0,1.0,2.0,116430.75,4630.50,0,2,...,2,1,0,0,0,0,1,0,0,14
80897,54000.0,1.0,0.0,1.0,1.0,0.0,52128.00,2742.30,0,1,...,0,0,0,1,0,0,1,0,1,0
80898,13500.0,0.0,0.0,1.0,0.0,0.0,25470.00,1435.05,0,1,...,0,1,0,1,0,0,1,0,0,8


In [220]:
df = df.sort_index(axis=1)

In [221]:
pt = PowerTransformer(method = 'yeo-johnson', standardize = True, copy = False)
df = pt.fit_transform(df)

In [222]:
with open(r"E:\data science\NBFI project/model.pkl", 'rb') as file:
                loaded_model = pickle.load(file)
with open(r'E:\data science\NBFI project/scaler.pkl', 'rb') as f:
                scaler_loaded = pickle.load(f)


In [223]:
new_sample = scaler_loaded.transform(df)

In [224]:
new_pred = loaded_model.predict(new_sample)
Default = pd.DataFrame(new_pred)
Default.value_counts()



0    80839
1       60
Name: count, dtype: int64

In [225]:
df3 = pd.concat([data, Default], axis=1)
rows = df3[df3[0] == 1]
rows


,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,...,new_Client_Education,new_Client_Marital_Status,new_Client_Gender,new_Loan_Contract_Type,new_Client_Housing_Type,new_Application_Process_Day,new_Client_Permanent_Match_Tag,new_Client_Contact_Work_Tag,new_Type_Organization,0
4146,12249577.0,11250.0,0.0,1.0,1.0,1.0,0.0,45000.00,2310.75,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,17.0,1.0
7050,12211177.0,15750.0,1.0,0.0,1.0,1.0,1.0,52128.00,3539.25,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
18400,12231713.0,15750.0,0.0,0.0,1.0,1.0,0.0,51477.75,2486.70,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
18953,12214731.0,13500.0,0.0,0.0,0.0,1.0,0.0,51477.75,2486.70,4.0,...,1.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0
19049,12208710.0,15750.0,1.0,0.0,0.0,1.0,1.0,51477.75,2486.70,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
19192,12232962.0,18000.0,0.0,0.0,0.0,1.0,0.0,51477.75,2486.70,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
20665,12229127.0,12600.0,0.0,1.0,0.0,1.0,0.0,51477.75,2486.70,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
23535,12224118.0,11250.0,1.0,1.0,0.0,1.0,0.0,51477.75,2486.70,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
25265,12262669.0,18900.0,0.0,0.0,0.0,1.0,0.0,51477.75,2486.70,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
25553,12260770.0,14400.0,0.0,0.0,0.0,1.0,1.0,51477.75,2486.70,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
